In [2]:
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score

In [3]:
insp = pd.read_csv("inspections.csv")
network = pd.read_csv("network.csv")
samplesub = pd.read_csv("sample_submission.csv")

In [4]:
samplesub = samplesub.merge(network, how="left", on="PipeId")

In [5]:
samplesub.isna().sum()

PipeId                 0
Incidence              0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
dtype: int64

In [6]:
insp = insp.merge(network, how ="inner", on = "PipeId")

In [7]:
insp.isna().sum()

PipeId                 0
MaintenanceId          0
InspectionYear         0
InspectionDate         0
MonthsLastRev          0
Severity               0
Incidence              0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
dtype: int64

In [8]:
insp = insp.drop("InspectionDate", axis=1)
insp = insp.drop("MaintenanceId", axis=1)

In [9]:
insp["AgeWhenInspected"] = insp["InspectionYear"] - insp["YearBuilt"]

In [10]:
insp_pure = insp.drop(columns=["Province","Town","YearBuilt","Material","GasType","Diameter","Length","Pressure","NumConnections","NumConnectionsUnder","BoolBridle"])

In [11]:
latest_year_index = insp.groupby('PipeId')['InspectionYear'].idxmax()
filtered_df = insp.loc[latest_year_index]

In [12]:
samplesub = samplesub.drop("Incidence",axis=1)

In [13]:
insp = insp.drop(columns=["PipeId","InspectionYear"],axis=1)

In [14]:
filtered_df = filtered_df[["PipeId","InspectionYear"]]

In [15]:
samplesub = samplesub.merge(filtered_df, how="left", on="PipeId")

In [16]:
samplesub = samplesub.rename(columns={"InspectionYear": "PipeLastInspectionYear"})

In [17]:
samplesub

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,PipeLastInspectionYear
0,446859944,Valencia,Alginet,2012,PE,Gas natural,110.0,44.286,0.150,0,0,False,2019.0
1,428124500,Valencia,Alginet,2012,PE,Gas natural,160.0,110.715,0.150,2,0,False,2019.0
2,438428871,Valencia,Alginet,2012,PE,Gas natural,110.0,13.575,0.150,0,0,False,2019.0
3,429034569,Valencia,Alginet,2012,PE,Gas natural,110.0,327.689,0.150,3,0,False,2019.0
4,411184477,Valencia,Alginet,2007,PE,Gas natural,200.0,932.762,0.150,0,0,False,2019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,Valencia,Paterna,2004,PE,Gas natural,110.0,7.656,0.025,0,0,False,2020.0
909729,235426673,Valencia,Paterna,2004,PE,Gas natural,110.0,4.383,0.025,2,0,False,2020.0
909730,235426707,Valencia,Paterna,2004,PE,Gas natural,110.0,2.292,0.025,1,0,False,2020.0
909731,190956601,Valencia,Paterna,2004,PE,Gas natural,90.0,87.809,4.000,0,0,False,2020.0


In [18]:
samplesub.isna().sum()

PipeId                        0
Province                      0
Town                          0
YearBuilt                     0
Material                      0
GasType                       0
Diameter                      0
Length                        0
Pressure                      0
NumConnections                0
NumConnectionsUnder           0
BoolBridle                    0
PipeLastInspectionYear    18685
dtype: int64

In [19]:
samplesub["AgeWhenInspected"] = samplesub["PipeLastInspectionYear"] - samplesub["YearBuilt"]


In [20]:
samplesub = samplesub.drop("PipeLastInspectionYear",axis=1)

In [21]:
samplesub.isna().sum()

PipeId                     0
Province                   0
Town                       0
YearBuilt                  0
Material                   0
GasType                    0
Diameter                   0
Length                     0
Pressure                   0
NumConnections             0
NumConnectionsUnder        0
BoolBridle                 0
AgeWhenInspected       18685
dtype: int64

In [22]:
samplesub['AgeWhenInspected'] = samplesub['AgeWhenInspected'].fillna(0)

In [23]:
samplesub.isna().sum()

PipeId                 0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
AgeWhenInspected       0
dtype: int64

In [24]:
insp = insp.drop(columns=["MonthsLastRev","Severity"],axis=1)

In [25]:
insp.isna().sum()

Incidence              0
Province               0
Town                   0
YearBuilt              0
Material               0
GasType                0
Diameter               0
Length                 0
Pressure               0
NumConnections         0
NumConnectionsUnder    0
BoolBridle             0
AgeWhenInspected       0
dtype: int64

In [26]:
insp

,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,AgeWhenInspected
0,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,9
1,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,11
2,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,13
3,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,15
4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,0,Valencia,Betera,1993,PE,Gas natural,63.0,1.237,4.000,0,0,False,27
6345340,0,Valencia,Betera,2000,PE,Gas natural,160.0,169.732,4.000,1,0,False,20
6345341,0,Barcelona,Sabadell,1995,FD,Gas natural,150.0,0.361,0.025,0,0,True,26
6345342,0,Valencia,Betera,1993,PE,Gas natural,63.0,23.306,4.000,0,0,False,27


In [27]:
data = insp

In [28]:
data_y = data["Incidence"]

In [29]:
data = data.drop(columns="Incidence",axis=1)

In [30]:
sub = samplesub

In [31]:
sub

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle,AgeWhenInspected
0,446859944,Valencia,Alginet,2012,PE,Gas natural,110.0,44.286,0.150,0,0,False,7.0
1,428124500,Valencia,Alginet,2012,PE,Gas natural,160.0,110.715,0.150,2,0,False,7.0
2,438428871,Valencia,Alginet,2012,PE,Gas natural,110.0,13.575,0.150,0,0,False,7.0
3,429034569,Valencia,Alginet,2012,PE,Gas natural,110.0,327.689,0.150,3,0,False,7.0
4,411184477,Valencia,Alginet,2007,PE,Gas natural,200.0,932.762,0.150,0,0,False,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,Valencia,Paterna,2004,PE,Gas natural,110.0,7.656,0.025,0,0,False,16.0
909729,235426673,Valencia,Paterna,2004,PE,Gas natural,110.0,4.383,0.025,2,0,False,16.0
909730,235426707,Valencia,Paterna,2004,PE,Gas natural,110.0,2.292,0.025,1,0,False,16.0
909731,190956601,Valencia,Paterna,2004,PE,Gas natural,90.0,87.809,4.000,0,0,False,16.0


In [32]:
sub["Volume"] = ((3.14*(sub["Diameter"]/2000))**2*sub["Length"])
sub["VolumestimesPressure"] = sub["Volume"]*sub["Pressure"]
sub["VolumesdivPressure"] = sub["Volume"]/sub["Pressure"]
sub["Presstimesage"] = sub["Pressure"]*sub["AgeWhenInspected"]
sub["SurfaceArea"] = 2*3.1416*((sub["Diameter"]/2)**2) +  2*3.1416*((sub["Diameter"]/2)*sub["Length"])
sub["Circumference"] = 2*3.1416*(sub["Diameter"]/2)

sub["PresstimesSurf"] = sub["Pressure"] * sub["SurfaceArea"]
sub["PresstimesCircumference"] = sub["Pressure"] * sub["Circumference"] 
sub["AgesTimesLength"] = sub["AgeWhenInspected"] * sub["Length"]
sub["Connectionstimespressure"] = sub["NumConnections"] * sub["Pressure"]
sub["AgesTimesVol"] = sub["AgeWhenInspected"] * sub["Volume"]
sub["LenTimesDiameter"] = sub["Length"] * sub["Diameter"]

In [33]:
data["Volume"] = ((3.14*(data["Diameter"]/2000))**2*data["Length"])
data["VolumestimesPressure"] = data["Volume"]*data["Pressure"]
data["VolumesdivPressure"] = data["Volume"]/data["Pressure"]
data["Presstimesage"] = data["Pressure"]*data["AgeWhenInspected"]
data["SurfaceArea"] = 2*3.1416*((data["Diameter"]/2)**2) +  2*3.1416*((data["Diameter"]/2)*data["Length"])
data["Circumference"] = 2*3.1416*(data["Diameter"]/2)

data["PresstimesSurf"] = data["Pressure"] * data["SurfaceArea"]
data["PresstimesCircumference"] = data["Pressure"] * data["Circumference"] 
data["AgesTimesLength"] = data["AgeWhenInspected"] * data["Length"]
data["Connectionstimespressure"] = data["NumConnections"] * data["Pressure"]
data["AgesTimesVol"] = data["AgeWhenInspected"] * data["Volume"]
data["LenTimesDiameter"] = data["Length"] * data["Diameter"]

In [34]:
sub.to_csv("samplesubdefinitive.csv",index=False)
data.to_csv("datadefinitive.csv",index=False)
data_y.to_csv("data_ydefinitive.csv",index=False)

In [37]:
import optuna
import pandas as pd
import numpy as np
import xgboost as xgb
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score


In [38]:
#READ THE DATA
data = pd.read_csv("datadefinitive.csv")
data_y = pd.read_csv("data_ydefinitive.csv")
sub = pd.read_csv("samplesubdefinitive.csv")


In [ ]:
#TRANSFORM CATEGORICAL COLUMNS TO CATEGORY DATA TYPE
categorical_cols = ["Province","Town","Material","GasType","BoolBridle"]
sub[categorical_cols] = sub[categorical_cols].astype('category')
data[categorical_cols] = data[categorical_cols].astype('category')


In [ ]:
#SET PIPEID AS THE INDEX
sub = sub.set_index("PipeId")


In [ ]:
#TRAINT TEST SPLIT
X_train, X_test, y_train, y_test = train_test_split(data, data_y, test_size=0.1)


In [ ]:
#GROUP TOGETHER THE TRAINING DATA
fulltrain = pd.concat([X_train,y_train],axis=1)


In [ ]:
#PERFORM RANDOM UNDERSAMPLING
df = fulltrain.sample(frac=1)
incidence_df = fulltrain.loc[fulltrain['Incidence'] == 1]
non_incidence_df = fulltrain.loc[fulltrain['Incidence'] == 0][:11071]
normal_distributed_df = pd.concat([incidence_df, non_incidence_df])
new_df = normal_distributed_df.sample(frac=1, random_state=4


In [ ]:
#DEFINE TARGET AND DATAFRAME
new_df_y = new_df["Incidence"]
new_df_X = new_df.drop(columns="Incidence",axis=1)
numerical_cols = new_df_X.select_dtypes(include=['int', 'float']).columns



In [ ]:
#USING OPTUNA TO GET THE BEST PARAMS
def objective(trial):


    # Define the hyperparameters to search over
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000, step=50),
        'max_depth': trial.suggest_int('max_depth', 2, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'subsample': trial.suggest_float('subsample', 0.5, 1.5),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 20),
        'lambda': trial.suggest_int('lambda', 0.1, 1),
        'alpha': trial.suggest_int('alpha', 0.1, 0.5),        
        'num_paralell_trees': trial.suggest_int('num_paralell_trees', 1, 8),
        "enable_categorical":True,
        "tree_method":"gpu_hist"
    }
    
    # Train an XGBoost classifier with the specified hyperparameters
    clf = xgb.XGBClassifier(**params)
    clf.fit(new_df_X, new_df_y)
    
    # Calculate the auc on the test set
    y_pred = clf.predict(X_test)
    auc_res = roc_auc_score(y_test, y_pred)
    
    # Return the accuracy as the objective value for Optuna to optimize
    return auc_res


# Use Optuna to search for the best hyperparameters
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)

# Print the best hyperparameters found by Optuna
print(f'Best parameters: {study.best_params}')
print(f'Best auc: {study.best_value:.3f}')


In [ ]:
from xgboost import XGBClassifier
my_model = XGBClassifier(n_estimators= 800, max_depth= 9, learning_rate= 0.01707958992187511, subsample= 0.7710060008763321, colsample_bytree= 0.632414015040246, gamma= 1.6495326364643785, min_child_weight=8 ,enable_categorical=True,tree_method="gpu_hist")
my_model.fit(new_df_X, new_df_y)


In [ ]:
from xgboost import plot_importance
plot_importance(my_model)


In [ ]:
results_1 = my_model.predict_proba(sub)
results1df = pd.DataFrame(results_1[:,1])
ids = list(sub.index.values)  
results1df.index = ids


In [ ]:
results1df.rename(columns={results1df.columns[0]:'Incidence'}, inplace=True)
results1df


In [ ]:
results1df.to_csv("XGBoostFinalSub7.csv", index=True)
